In [4]:
import csv
import os
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# 0-读取csv数据，存成numpy

In [5]:
def data_load_from_path(filePath):
    feat_line = 0
    cur_feature_list = []
    with open(filePath,'r') as f:
        cur_file = csv.reader(f)
        for cur_feature in cur_file:
            if feat_line == 0:
                feat_line += 1
                continue
            cur_feature_line = []
            for f_data in cur_feature:
                cur_feature_line.append(float(f_data))
            # print(len(cur_feature_line))
            cur_feature_list.append(cur_feature_line)
            feat_line += 1
    return cur_feature_list

In [6]:
def write_file(file_path,file_name_list):
    ff = open(file_path,'w')
    for ii in file_name_list:
        ff.write(ii + "\n")
    ff.close()

In [7]:
def read_file(file_path):
    ff = open(file_path)
    file_name = []
    for ii in ff:
        file_name.append(ii.strip())
    ff.close()
    return file_name

In [8]:
def data_union(total_feature_list):
    maxLen = 0
    minLen = 2100000
    diffLenNumer = {}
    for data_array in total_feature_list:
        cur_len = len(data_array)
        if cur_len < minLen:
            minLen = cur_len
        if cur_len > maxLen:
            maxLen = cur_len
        if diffLenNumer.has_key(cur_len) == False:
            diffLenNumer[cur_len] = 1
        else:
            diffLenNumer[cur_len] = diffLenNumer[cur_len] + 1
    #for key in diffLenNumer.keys():
        #print(key," : ",diffLenNumer[key])
    print('minLen:',minLen," maxLen:",maxLen)
    maxLen = maxLen + 1
    data_numpy = np.zeros((len(total_feature_list),maxLen,len(total_feature_list[0][0])))
    for index,data_array in enumerate(total_feature_list):
        cur_array = np.array(data_array)
        data_numpy[index,maxLen - len(cur_array):,:] = cur_array[:,:]
        data_numpy[index][0][0] = maxLen - len(cur_array)
    print(data_numpy.shape)
    return data_numpy

In [9]:
tmpFile = r'./tmpFile/'
def data_load(trainPath,trainLabelpath,testPath=None):
    csv_file = csv.reader(open(trainLabelpath,'r'))
    cnt = 0
    labelList = []
    total_feature_list = []
    for stu in csv_file:
        if cnt % 1000 == 2:
            print(cnt)
        if cnt == 0:
            cnt += 1
            continue
        cur_label = int(stu[2])
        cur_path = trainPath + stu[0] + '/' + stu[1]
        if os.path.exists(cur_path):
            cur_feature_list = data_load_from_path(cur_path)
            total_feature_list.append(cur_feature_list)
            labelList.append(cur_label)
        cnt += 1
    labelList = np.array(labelList)
    trainData = data_union(total_feature_list)
    total_feature_list_test = []
    testFileName = []
    for index,cur_path in enumerate(os.listdir(testPath)):
        if index % 1000 == 2:
            print(index)
        testFileName.append(cur_path)
        cur_feature_list = data_load_from_path(testPath + cur_path)
        total_feature_list_test.append(cur_feature_list)
    testData = data_union(total_feature_list_test)
    if os.path.exists(tmpFile) == False:
        os.mkdir(tmpFile)
    np.save(tmpFile + 'trainData.npy',trainData)
    np.save(tmpFile + 'labelList.npy',labelList)
    np.save(tmpFile + 'testData.npy',testData)
    write_file(tmpFile + 'test_name.txt',testFileName)
    return trainData,labelList,testData,testFileName

### 读取数据，这部分建议在服务器上跑，以免出现内存或者硬盘空间不足的情况。numpy文件将保存到tmpFile目录下。

In [ ]:
rootpath = r'./'
trainPath = rootpath + 'train/'
testPath = rootpath + 'test/'
labelPath = rootpath + 'train_labels.csv'
trainX,trainY,testX,testFileName = data_load(trainPath,labelPath,testPath)

2
1002
2002
3002
4002
5002
6002
7002
8002
9002
10002
11002
12002
13002
14002
15002
16002
17002
18002
19002
20002
21002
22002
23002
24002
25002
26002


# 1-数据处理

## 数据加载

In [10]:
trainData = np.load(tmpFile + 'trainData.npy')
labelList = np.load(tmpFile + 'labelList.npy')
testData = np.load(tmpFile + 'testData.npy')
testFileName = read_file(tmpFile + 'test_name.txt')

In [11]:
print trainData.shape,labelList.shape,testData.shape,len(testFileName)

(48339, 451, 75) (48339,) (87498, 451, 75) 87498


### 删除最后三列

In [12]:
trainData = trainData[:,:,:-3]
testData = testData[:,:,:-3]
print trainData.shape,testData.shape

(48339, 451, 72) (87498, 451, 72)


### 数据处理，删除轮毂转速（index-0）为0且超速传感器转速检测值（indx-8）为0的行。如果test-data的当前文件全部为0，则直接判定为0，并保存到delete-index

In [13]:
def clean_speed0(data,labelList=None,data_name=None,train_flag=True):
    maxLen = len(data[0])
    factor_number = len(data[0][0])
    clean_data_item0 = []
    clean_y = []
    clean_name = []
    delete_index = []
    for ii in range(len(data)):
        st = int(data[ii][0][0])
        cur_data = data[ii,st:]
        speed_0 = np.where((cur_data[:,0] == 0) & (cur_data[:,8] == 0))[0]  ##speed 0 is deleted
        cur_data = np.delete(cur_data, speed_0, 0)
        if len(cur_data) < 1:
            delete_index.append(ii)
            continue
        clean_item0_equal_0 = np.zeros((maxLen,factor_number))
        new_st = maxLen - len(cur_data)
        clean_item0_equal_0[new_st:,:] = cur_data
        clean_item0_equal_0[0][0] = new_st
        clean_data_item0.append(clean_item0_equal_0)
        if train_flag:
            clean_y.append(labelList[ii])
        else:
            clean_name.append(data_name[ii])
    clean_data_item0_array = np.array(clean_data_item0)
    #print clean_data_item0_array.shape
    return  clean_data_item0_array,np.array(clean_y),clean_name,delete_index

In [14]:
clean_train_data,clean_train_y,_,train_delete_index = clean_speed0(trainData,labelList=labelList)

In [15]:
print trainData.shape,clean_train_data.shape,len(clean_train_y),len(train_delete_index)

(48339, 451, 72) (46620, 451, 72) 46620 1719


In [16]:
## 避免内存占用过度，删除trainData
del trainData

In [17]:
clean_test_data,_,clean_test_name,test_delete_index = clean_speed0(testData,data_name=testFileName,train_flag=False)

In [18]:
print testData.shape,clean_test_data.shape,len(clean_test_name),len(test_delete_index)

(87498, 451, 72) (87139, 451, 72) 87139 359


In [19]:
## 避免内存占用过度，删除testData
del testData

In [20]:
np.save(tmpFile + 'clean_train_data.npy',clean_train_data)
np.save(tmpFile + 'clean_train_y.npy',clean_train_y)
np.save(tmpFile + 'clean_test_data.npy',clean_test_data)
write_file(tmpFile + 'clean_test_name.txt',clean_test_name)
np.save(tmpFile + 'delete_index.npy',test_delete_index)  ##test_index should be saved

# stage-1 整体--数据提取特征和F1指标训练

In [53]:
def feature_mean_and_std(data_array):
    if len(data_array.shape) == 1:
        data_array = data_array.reshape(-1,1)
    feature_time_and_fft = []
    feature_time_and_fft.extend(np.mean(data_array,axis=0))
    feature_time_and_fft.extend(np.std(data_array,axis=0))
    return feature_time_and_fft

### 特征1——提取所有变量的均值和方差，共计72 * 2 = 144维特征

In [22]:
def get_feature(data):
    item0_fea = []
    for ii in range(len(data)):
        st = int(data[ii][0][0])
        cur_data = data[ii,st:]
        item0_fea.append(feature_mean_and_std(cur_data))
    item0_ff = np.array(item0_fea)
    return item0_ff

In [23]:
trainX = get_feature(clean_train_data)

In [24]:
testX = get_feature(clean_test_data)

In [25]:
print trainX.shape,clean_train_y.shape,testX.shape,len(clean_test_name)

(46620, 144) (46620,) (87139, 144) 87139


### 特征2——对index-65提取特征

In [46]:
def feature_item65(data):
    data65 = data[:,-100:,65]
    feature = []
    for ii in range(len(data65)):
        pt = np.where(data65[ii] > 100)[0]
        if len(pt) > 0:
            feature.append(1)
        else:
            feature.append(0)
    feature = np.array(feature)
    #print feature.shape
    return feature

In [47]:
train_fea_item65 = feature_item65(clean_train_data)

In [48]:
print len(np.where(train_fea_item65 == 0)[0]),len(np.where(train_fea_item65 == 1)[0])

32683 13937


In [49]:
test_fea_item65 = feature_item65(clean_test_data)

In [50]:
print train_fea_item65.shape,test_fea_item65.shape

(46620,) (87139,)


In [51]:
print len(np.where(test_fea_item65 == 0)[0]),len(np.where(test_fea_item65 == 1)[0])

76287 10852


### 通过上述对index-65的划分，我们可以看到train集合和test集合是有差别的。

## 特征3——对下面的数据进行处理，避免左偏，右偏现象

In [56]:
maxLen = 451
feature_number = 2
def new_item_index(data):  ##power(x,0.5)
    power_index = (13,19,20,45,46,21,23)
    new_item_index_feature = []
    for ii in range(len(data)):
        st = int(data[ii][0][0])
        cur_data = data[ii,st:,:]
        cur_data_fea = []
        for jj in range(len(power_index)):
            part_data = cur_data[:,power_index[jj]]
            equal_0 = np.where(part_data == 0)[0]
            part_data = np.delete(part_data, equal_0, 0)
            if len(part_data) == 0:
                cur_data_fea.extend(np.zeros(feature_number))
            else:
                new_part_data = np.sign(part_data) * np.sqrt(np.abs(part_data))
                new_part_data = np.array(new_part_data)
                #new_part_data = np.power(part_data,0.5)
                #new_part_data = np.log(part_data)
                cur_data_fea.extend(feature_mean_and_std(new_part_data))
            cur_data_fea.append(len(equal_0) * 1.0 / (maxLen - st))
        new_item_index_feature.append(cur_data_fea)
    feature = np.array(new_item_index_feature)
    print feature.shape
    return feature

In [57]:
train_fea_new_item_power = new_item_index(clean_train_data)

(46620, 21)


In [58]:
test_fea_new_item_power = new_item_index(clean_test_data)

(87139, 21)


In [59]:
## 检测是否存在nan
print np.isnan(np.sum(train_fea_new_item_power))
print np.isnan(np.sum(test_fea_new_item_power))

False
False


### 特征拼接，得到166维特征向量

In [65]:
train_fea_item65 = train_fea_item65.reshape(-1,1)
test_fea_item65 = test_fea_item65.reshape(-1,1)

In [66]:
print trainX.shape,train_fea_item65.shape,train_fea_new_item_power.shape

(46620, 144) (46620, 1) (46620, 21)


In [67]:
trainX = np.concatenate((trainX,train_fea_item65,train_fea_new_item_power),axis=1)

In [68]:
testX = np.concatenate((testX,test_fea_item65,test_fea_new_item_power),axis=1)

In [69]:
print trainX.shape,testX.shape

(46620, 166) (87139, 166)


### 特征preprocessing与训练

In [70]:
from sklearn import preprocessing
def data_preprocessing(trainX,testX):
    #scaler = preprocessing.RobustScaler(quantile_range=(10.0, 90.0)).fit(trainX)
    scaler = preprocessing.StandardScaler().fit(trainX)
    scaler.transform(trainX)
    scaler.transform(testX)
    return trainX,testX

In [71]:
trainX,testX = data_preprocessing(trainX,testX)

In [72]:
def define_model():
    classifiers = {
      #'RF': RandomForestClassifier(n_estimators=100, max_depth=10, max_features=0.8,n_jobs=12,verbose=1),  # clf.feature_importances_
      'GB': GradientBoostingClassifier(n_estimators=30, learning_rate=0.001, max_depth=5, max_features=0.5,verbose=0,random_state=0),
    }
    return classifiers

In [73]:
def model_select_cross_val_score(X,y): ##
    classifiers = define_model()
    best_clf = classifiers.items()[0]
    best_sc = 0.0
    for name, clf in classifiers.items():
        scores = cross_val_score(clf, X, y, cv=5, scoring='f1',n_jobs=12)
        sc_av = scores.mean()
        print(name, '\t--> ', sc_av)
        if best_sc < sc_av:
            best_sc = sc_av
            best_clf = clf
    print(best_sc)
    return best_clf

In [74]:
clf = model_select_cross_val_score(trainX,clean_train_y)

('GB', '\t--> ', 0.6507631392285147)
0.6507631392285147


In [75]:
X_train, X_test, y_train, y_test = train_test_split(trainX,clean_train_y, test_size=0.5,random_state=1234)
clf.fit(X_train, y_train)
predict_t = clf.predict(X_test)  ##predict_proba
f1_sc = metrics.f1_score(predict_t, y_test)
print("f1:",f1_sc)
acc = metrics.accuracy_score(predict_t, y_test)
print("acc:",acc)

('f1:', 0.7343659618227392)
('acc:', 0.6453882453882454)


In [76]:
print len(clean_train_y[np.where(predict_t == 0)[0]]),len(clean_train_y[np.where(predict_t == 1)[0]])

4053 19257


In [77]:
clf.fit(trainX, clean_train_y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.001, loss='deviance', max_depth=5,
              max_features=0.5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [78]:
predict_label = clf.predict(testX)
print predict_label.shape
print len(test_delete_index),len(testX),len(testFileName),len(clean_test_name)

(87139,)
359 87139 87498 87139


### 至此，将得到第一阶段的预测结果文件67053.csv，线上F1: 0.67053

In [80]:
outPath = tmpFile + '/67053.csv'
with open(outPath,"w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id","ret"])
    for index,cur_label in enumerate(predict_label):
        writer.writerow([clean_test_name[index],str(cur_label)])
    for ii in range(len(test_delete_index)):
        index = test_delete_index[ii]
        writer.writerow([testFileName[index],'0'])

# stage-2 数据划分--数据提取新特征和accuracy指标训练

## 特征4——提取，除了index-8和index-15外的其他index除index-0的向量的均值和方差。

In [82]:
maxLen = 451
feature_number = 2
def new_item0(data):  ##power(x,0.5)
    new_item_index_feature = []
    for ii in range(len(data)):
        st = int(data[ii][0][0])
        cur_data = data[ii,st:,:]
        cur_data_fea = []
        feature_number = len(cur_data[0])
        speed = []
        for index in range(len(cur_data)):
            if cur_data[index,0] == 0:
                speed.append(cur_data[index,8])
            else:
                speed.append(cur_data[index,0])
        speed = np.array(speed)
        if len(np.where(speed == 0)[0]) > 0:
            print "error.",ii
        for jj in range(1,len(cur_data[0])):
            if jj == 8 or jj == 15:
                continue
            part_data = cur_data[:,jj]
            new_part_data = part_data / speed
            #new_part_data = np.power(part_data,0.5)
            #new_part_data = np.log(part_data)
            cur_data_fea.extend(feature_mean_and_std(new_part_data))
        new_item_index_feature.append(cur_data_fea)
    feature = np.array(new_item_index_feature)
    print feature.shape
    return feature

In [83]:
item0_feature = new_item0(clean_train_data)

(46620, 138)


In [84]:
item0_feature_test = new_item0(clean_test_data)

(87139, 138)


In [85]:
## 检测是否存在nan
print np.isnan(np.sum(item0_feature))
print np.isnan(np.sum(item0_feature_test))

False
False


## 特征5——所有变量的时序特征（fft后的均值和方差）

In [86]:
def time_feature(sequence_data):
    fft_trans = np.abs(np.fft.fft(sequence_data))
    dc = fft_trans[0]
    freq_spectrum = fft_trans[1:int(np.floor(len(sequence_data) * 1.0 / 2)) + 1]
    _freq_sum_ = np.sum(freq_spectrum)
    time_feature = []
    time_feature.append(np.mean(freq_spectrum))
    time_feature.append(np.std(freq_spectrum))
    return time_feature

In [87]:
maxLen = 451
feature_number = 2
def get_time_feature(data):  
    new_item_index_feature = []
    #feature_index = [0,1,2,3,4,8,16,18,27,28]
    feature_index = range(0,72)
    for ii in range(len(data)):
        st = int(data[ii][0][0])
        cur_data = data[ii,st:,:]
        cur_data_fea = []
        for jj in range(len(feature_index)):
            sequence_data = cur_data[:,feature_index[jj]]
            cur_data_fea.extend(time_feature(sequence_data))
        new_item_index_feature.append(cur_data_fea)
    feature = np.array(new_item_index_feature)
    print feature.shape
    return feature

In [88]:
train_time_fea = get_time_feature(clean_train_data)

/usr/local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(46620, 144)


In [89]:
print np.isnan(np.sum(train_time_fea))

True


In [92]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from matplotlib import pyplot

/usr/local/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [93]:
def deal_nan(data):
    df = DataFrame(data)
    print np.any(df.isnull()) == True
    #df = df.fillna(value=0) 
    df = df.fillna(df.mean())
    print np.any(df.isnull()) == True
    data = np.array(df)
    print np.isnan(np.sum(data))
    return data

In [94]:
train_time_fea = deal_nan(train_time_fea)

True
False
False


In [95]:
test_time_fea = get_time_feature(clean_test_data)

(87139, 144)


In [96]:
test_time_fea = deal_nan(test_time_fea)

True
False
False


In [97]:
## 检测是否存在nan
print np.isnan(np.sum(item0_feature))
print np.isnan(np.sum(item0_feature_test))

False
False


### 特征拼接，得到448维特征向量

In [98]:
trainX = np.concatenate((trainX,item0_feature,train_time_fea),axis=1)

In [99]:
testX = np.concatenate((testX,item0_feature_test,test_time_fea),axis=1)

In [100]:
print trainX.shape,testX.shape

(46620, 448) (87139, 448)


In [101]:
np.save(tmpFile + '/train_feature448.npy',trainX)
np.save(tmpFile + '/test_feature448.npy',testX)

## 数据清洗，删除train，test不同分布的数据

In [102]:
train_fea = np.load(tmpFile + '/train_feature448.npy')
test_fea = np.load(tmpFile + '/test_feature448.npy')

In [105]:
del_index_item16 = np.where((train_fea[:,16 + 72] != 0))[0]
del_index_item55 = np.where((train_fea[:,55] != 4))[0]
del_index_item00 = np.where((train_fea[:,0] < 0))[0]
del_i = []
del_i.extend(del_index_item16)
del_i.extend(del_index_item55)
del_i.extend(del_index_item00)
print len(del_i)
del_i = np.unique(np.array(del_i))
print len(del_i), len(del_i) * 1.0 / len(train_fea)

11412
10524 0.22574002574


In [106]:
del_index_item0 = np.where((test_fea[:,16 + 72] != 0))[0]
del_index_item1 = np.where((test_fea[:,55] != 4))[0]
del_index_item2 = np.where((test_fea[:,0] < 0))[0]
del_j = []
del_j.extend(del_index_item0)
del_j.extend(del_index_item1)
del_j.extend(del_index_item2)
print len(del_j)
del_j = np.unique(np.array(del_j))
print len(del_j), len(del_j) * 1.0 / len(test_fea)

5820
5014 0.0575402517816


In [107]:
print len(np.where(clean_train_y == 0)[0]),len(np.where(clean_train_y == 1)[0])

23030 23590


In [108]:
print len(np.where(clean_train_y[del_i] == 0)[0]),len(np.where(clean_train_y[del_i] == 1)[0])

4535 5989


In [109]:
train_fea_clean = np.delete(train_fea,del_i,axis=0)
train_y_clean = np.delete(clean_train_y,del_i,axis=0)

In [110]:
print train_fea_clean.shape,train_y_clean.shape

(36096, 448) (36096,)


In [111]:
test_fea_clean = np.delete(test_fea,del_j,axis=0)
print test_fea_clean.shape

(82125, 448)


In [113]:
clean_test_name2 = []
for ii in range(len(clean_test_name)):
    if ii not in del_j:
        clean_test_name2.append(clean_test_name[ii])
print len(clean_test_name2)
clean_test_name = clean_test_name2

82125


### 数据划分--36part，根据index-53,index-54,index65,index-16，分别代表机器的不同状态

In [115]:
data_split_train_index = []
data_split_test_index = []
item53 = (6,3,2)
item54 = (6,4)
for ii in range(1,4):  ##16
    for jj in range(0,2): ##65,
        for kk in range(len(item53)): ##53
            for LL in range(len(item54)):##54
                if jj == 0:
                    index = np.where((train_fea_clean[:,16] == ii) & (train_fea_clean[:,65] < 300) 
                                     & (train_fea_clean[:,53] == item53[kk]) &
                                       (train_fea_clean[:,54] == item54[LL]))[0]
                else:
                    index = np.where((train_fea_clean[:,16] == ii) & (train_fea_clean[:,65] >= 300) 
                                     & (train_fea_clean[:,53] == item53[kk]) &
                                       (train_fea_clean[:,54] == item54[LL]))[0]
                data_split_train_index.append(index)
                if len(index) == 0:
                    pass
                    #print ii,jj,item53[kk],item54[LL]
                if jj == 0:
                    index_test = np.where((test_fea_clean[:,16] == ii) & (test_fea_clean[:,65] < 300) 
                                     & (test_fea_clean[:,53] == item53[kk]) &
                                       (test_fea_clean[:,54] == item54[LL]))[0]
                else:
                    index_test = np.where((test_fea_clean[:,16] == ii) & (test_fea_clean[:,65] >= 300) 
                                     & (test_fea_clean[:,53] == item53[kk]) &
                                       (test_fea_clean[:,54] == item54[LL]))[0]
                data_split_test_index.append(index_test)
                
print len(data_split_train_index)

36


#### 查看每个划分下的数据情况

In [116]:
sumV = 0
for ii in range(len(data_split_test_index)):
    if len(data_split_test_index[ii]) < 10:
        continue
    print ii,len(data_split_test_index[ii])
    sumV += len(data_split_test_index[ii])
print sumV,sumV * 1.0 / len(test_fea_clean)

0 18665
3 2764
5 781
6 3847
9 302
11 37
12 94
15 832
17 2382
21 62
23 148
24 39356
27 2920
29 958
30 4937
33 159
35 119
78363 0.954191780822


In [117]:
sumV = 0
for ii in range(len(data_split_train_index)):
    cur_part = data_split_train_index[ii]
    if len(cur_part) < 10:
        continue
    print len(data_split_test_index[ii]),ii,len(cur_part),len(np.where(train_y_clean[cur_part] == 0)[0]),len(np.where(train_y_clean[cur_part] == 1)[0])
    sumV += len(data_split_train_index[ii])
print sumV,sumV * 1.0 / len(train_fea_clean)

18665 0 7227 4240 2987
2764 3 1094 726 368
781 5 585 215 370
3847 6 8197 4293 3904
302 9 1016 640 376
37 11 172 112 60
94 12 48 36 12
832 15 295 177 118
2382 17 268 199 69
62 21 100 33 67
39356 24 11712 5201 6511
2920 27 873 370 503
958 29 195 150 45
4937 30 1844 784 1060
159 33 92 26 66
33718 0.934120124113


### 从上述结果，我们下面着重分析子part0和part24

## 子part24，train model

In [172]:
index = 24
partIndex = data_split_train_index[index]
train_fea_clean_part = train_fea_clean[partIndex]
train_y_clean_part = train_y_clean[partIndex]
print train_fea_clean_part.shape,train_y_clean_part.shape

(11712, 448) (11712,)


In [173]:
test_index_part = data_split_test_index[index]
test_fea_clean_part = test_fea_clean[test_index_part]
print test_fea_clean_part.shape

(39356, 448)


In [174]:
#train_y_clean_part,test_fea_clean_part = data_preprocessing(train_fea_clean_part,test_fea_clean_part)

In [213]:
def model_select_cross_val_score(X,y): ##
    classifiers = define_model()
    best_clf = classifiers.items()[0]
    best_sc = 0.0
    for name, clf in classifiers.items():
        scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy',n_jobs=12)  ##accuracy,f1_weighted
        sc_av = scores.mean()
        print scores
        print scores.std()
        print(name, '\t--> ', sc_av)
        if best_sc < sc_av:
            best_sc = sc_av
            best_clf = clf
    print(best_sc)
    return best_clf

In [214]:
def define_model():
    classifiers = {
      'RF': RandomForestClassifier(n_estimators=30, max_depth=4, max_features=0.5,n_jobs=12,verbose=1,
                                   random_state=1234,class_weight={0:1,1:2}),  # clf.feature_importances_
    }
    return classifiers

In [215]:
clf = model_select_cross_val_score(train_fea_clean_part,train_y_clean_part)

[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:   13.9s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:   13.8s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:   13.7s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:   13.4s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:   12.6s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


[0.55375427 0.6118702  0.58625107 0.53928266 0.70025619]
0.05690651769978695
('RF', '\t--> ', 0.5982828771535079)
0.5982828771535079


In [216]:
X_train, X_test, y_train, y_test = train_test_split(train_fea_clean_part,train_y_clean_part,test_size=0.2,random_state=1234)
clf.fit(X_train, y_train)
predict_t = clf.predict(X_test)  ##predict_proba

[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    2.8s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


In [217]:
print len(np.where(predict_t == 0)[0]),len(np.where(predict_t == 1)[0])
mid_index = len(np.where(predict_t == 0)[0]) // 3

405 1938


In [218]:
predict_t_c = clf.predict_proba(X_test)  ##predict_proba
predict_t_xx = np.ones(len(predict_t_c))
sort_index_predict = np.argsort(-predict_t_c[:,0])[:mid_index]   ##等于上述1764的一半，然后用这个概率卡test集的阈值
#print predict_t_c[sort_index_predict][:,0],y_test[sort_index_predict]
print predict_t_c[sort_index_predict[-1]]
predict_t_xx[sort_index_predict] = 0
acc = metrics.accuracy_score(predict_t_xx[sort_index_predict], y_test[sort_index_predict])
print("acc-label0:",acc)
threshold = predict_t_c[sort_index_predict[-1]][0]

[0.86537439 0.13462561]
('acc-label0:', 1.0)


[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


In [219]:
clf.fit(train_fea_clean_part,train_y_clean_part)

[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    3.7s finished


RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 2},
            criterion='gini', max_depth=4, max_features=0.5,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=12, oob_score=False, random_state=1234,
            verbose=1, warm_start=False)

In [220]:
part_predict_proba = clf.predict_proba(test_fea_clean_part) 
print part_predict_proba.shape

(39356, 2)


[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


In [221]:
score_sort = np.argsort(-part_predict_proba[:,0])[:2000]

In [222]:
print np.max(part_predict_proba[score_sort,0]),np.min(part_predict_proba[score_sort,0])

0.9958094501346794 0.7558864903915677


In [223]:
part_predict = np.ones(len(part_predict_proba))
print len(part_predict)
part_predict[score_sort] = 0

39356


In [224]:
print len(np.where(part_predict == 0)[0]),len(np.where(part_predict == 1)[0])

2000 37356


In [225]:
print test_index_part.shape
print len(clean_test_name)
print part_predict_proba.shape

(39356,)
82125
(39356, 2)


In [226]:
mm = {}
for ii in range(len(test_index_part)):
    if part_predict[ii] == 0:
        label0_name = clean_test_name[test_index_part[ii]]
        mm[label0_name] = part_predict_proba[ii][0]
print len(mm.keys())

2000


### 至此,得到67234.csv,线上分数0.67234

In [227]:
cnt = 0
with open(tmpFile + '/67234.csv','w') as ff:
    with open(tmpFile + '/67053.csv') as readLine:
        for indextt,line in enumerate(readLine):
            if indextt == 0:
                ff.write(line)
            else:
                tt = line.strip().split(',')
                if int(tt[1]) == 0:
                    ff.write(tt[0] + ",0\n")
                else:
                    if mm.has_key(tt[0]):
                        cnt += 1
                        ff.write(tt[0] + ",0\n")
                    else:
                        ff.write(tt[0] + ",1\n")
print cnt

2000


## 子part0，train model

In [243]:
index = 0
partIndex = data_split_train_index[index]
train_fea_clean_part = train_fea_clean[partIndex]
train_y_clean_part = train_y_clean[partIndex]
print train_fea_clean_part.shape,train_y_clean_part.shape

(7227, 448) (7227,)


In [244]:
test_index_part = data_split_test_index[index]
test_fea_clean_part = test_fea_clean[test_index_part]
print test_fea_clean_part.shape

(18665, 448)


In [257]:
clf = RandomForestClassifier(n_estimators=30, max_depth=5, max_features=0.5,n_jobs=12,verbose=1,
                                   random_state=1234,class_weight={0:1,1:2})

In [258]:
#clf = GradientBoostingClassifier(n_estimators=30, learning_rate=0.001,max_depth=5,max_features=1.0,verbose=1,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(train_fea_clean_part,train_y_clean_part,test_size=0.5,random_state=1234)
clf.fit(X_train, y_train)
predict_t = clf.predict(X_test)  ##predict_proba

[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    1.2s finished
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


In [259]:
print len(np.where(predict_t == 0)[0]),len(np.where(predict_t == 1)[0])
mid_index = len(np.where(predict_t == 0)[0]) // 3

1805 1809


In [260]:
predict_t_c = clf.predict_proba(X_test)  ##predict_proba
predict_t_xx = np.ones(len(predict_t_c))
sort_index_predict = np.argsort(-predict_t_c[:,0])[:mid_index]   ##等于上述1764的一半，然后用这个概率卡test集的阈值
#print predict_t_c[sort_index_predict][:,0],y_test[sort_index_predict]
print predict_t_c[sort_index_predict[-1]]
predict_t_xx[sort_index_predict] = 0
acc = metrics.accuracy_score(predict_t_xx[sort_index_predict], y_test[sort_index_predict])
print("acc-label0:",acc)
threshold = predict_t_c[sort_index_predict[-1]][0]

[0.94095847 0.05904153]
('acc-label0:', 1.0)


[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


In [261]:
clf.fit(train_fea_clean_part,train_y_clean_part)

[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    2.5s finished


RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 2},
            criterion='gini', max_depth=5, max_features=0.5,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=12, oob_score=False, random_state=1234,
            verbose=1, warm_start=False)

In [262]:
part_predict_proba = clf.predict_proba(test_fea_clean_part) 
print part_predict_proba.shape

[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:    0.0s finished


(18665, 2)


In [263]:
score_sort = np.argsort(-part_predict_proba[:,0])[:2500]

In [264]:
print np.max(part_predict_proba[score_sort,0]),np.min(part_predict_proba[score_sort,0])

0.9731655332587609 0.8768205959068534


In [265]:
part_predict = np.ones(len(part_predict_proba))
print len(part_predict)
part_predict[score_sort] = 0

18665


In [266]:
print len(np.where(part_predict == 0)[0]),len(np.where(part_predict == 1)[0])

2500 16165


In [267]:
mm = {}
for ii in range(len(test_index_part)):
    if part_predict[ii] == 0:
        label0_name = clean_test_name[test_index_part[ii]]
        mm[label0_name] = part_predict_proba[ii][0]
print len(mm.keys())

2500


### 至此,得到67835.csv,线上分数0.67835

In [268]:
cnt = 0
with open(tmpFile + '/67835.csv','w') as ff:
    with open(tmpFile + '/67234.csv') as readLine:
        for indextt,line in enumerate(readLine):
            if indextt == 0:
                ff.write(line)
            else:
                tt = line.strip().split(',')
                if int(tt[1]) == 0:
                    ff.write(tt[0] + ",0\n")
                else:
                    if mm.has_key(tt[0]):
                        cnt += 1
                        ff.write(tt[0] + ",0\n")
                    else:
                        ff.write(tt[0] + ",1\n")
print cnt

2484
